# TimeAttentionレイヤを実装する

In [1]:
import numpy as np
from common.layers import Softmax

### [演習]
* 以下のWeightSum, AttentionWeight, Attention, TimeAttentionクラスを完成させましょう

In [2]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [], []
        self.cache = None

    def forward(self, hs, a):
        """
        順伝播
        hs : エンコーダの中間状態
        a : アテンション荷重
        """
        N, T, H = hs.shape

        # アテンション荷重の行列を3次元配列に変形する
        ar = a.reshape(N, T, 1)#.repeat(H, axis=2)   ブロードキャストを明示的に行いたい場合はrepeatを付ける
        # エンコーダの中間状態にアテンション荷重をかけて、それを足し合わせることによって、加重平均を求める
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)
        return c  # エンコーダの中間状態を加重平均した結果

    def backward(self, dc):
        """
        逆伝播
        """
        hs, ar = self.cache
        N, T, H = hs.shape
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)

        return dhs, da


class AttentionWeight:
    """
    アテンション荷重を算出するクラス
    """
    def __init__(self):
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None

    def forward(self, hs, h):
        """
        順伝播
        アテンション荷重を求める
        hs : エンコーダの全ての中間状態
        h : デコーダのある場所の中間状態
        """
        N, T, H = hs.shape

        #　デコーダのある場所の中間状態を3次元配列に変形する
        hr = h.reshape(N, 1, H)#.repeat(T, axis=1)
        
        # エンコーダの中間状態とデコーダの中間状態を掛けて足し合わせることで内積をとる
        # 他の実装例として、hsとhrを結合し、重みWを掛けるという方法もある
        t = hs * hr
        s = np.sum(t, axis=2)
        
        # ソフトマックス関数に通すことで、正規化する
        a = self.softmax.forward(s) # アテンション重みベクトルを並べた行列 (N * T)

        self.cache = (hs, hr)
        return a

    def backward(self, da):
        """
        逆伝播
        """
        hs, hr = self.cache
        N, T, H = hs.shape

        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh


class Attention:
    """
    アテンション
    """
    def __init__(self):
        self.params, self.grads = [], []
        
        # レイヤの定義
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        """
        順伝播
        hs : エンコーダの中間状態
        h : デコーダの中間状態
        """
        # アテンション荷重を求める
        a = self.attention_weight_layer.forward(hs, h)
        
        # エンコーダの中間状態にアテンション荷重をかける
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        
        return out # エンコーダの中間状態を加重平均した結果

    def backward(self, dout):
        """
        逆伝播
        """
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh


class TimeAttention:
    """
    アテンションレイヤを時間方向にまとめるレイヤ
    """
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        """
        順伝播
        hs_enc : エンコーダの中間状態
        hs_dec : デンコーダの中間状態
        """
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            """
            出力単語数分を繰り返す
            """
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:]) 
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        """
        逆伝播
        dout : 勾配
        """
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            """
            出力単語数分を繰り返す
            """
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, dhs_dec


In [3]:
# 中間層ノード数
H = 4
# データ数
N = 3
# 単語数
T = 5


# モデル構築
ta = TimeAttention()

hs_enc = np.random.randn(N*T*H).reshape(N, T, H)
hs_dec =  np.random.randn(N*T*H).reshape(N, T, H)
print("hs_enc=", hs_enc)
print()
print("hs_dec=", hs_dec)
print()

# 順伝播計算
out = ta.forward(hs_enc, hs_dec)
print("out=", out)
print()

# 逆伝播計算
dout = np.random.randn(N*T*H).reshape(N, T, H)
dhs_enc, dhs_dec = ta.backward(dout)
print("dhs_enc=", dhs_enc)
print()
print("dhs_dec=", dhs_dec)
print()

hs_enc= [[[-1.02082814  0.55732175  0.22309253 -0.27049653]
  [ 1.08273328  0.37609399 -2.62666916  2.31467079]
  [-0.49846795  0.69727866  0.60285361  0.9012636 ]
  [ 1.64457397 -0.84951114  0.7382962   2.16226523]
  [-0.27046016 -0.99635624 -0.31343818 -0.10097855]]

 [[ 0.08873106 -1.87342375 -2.10518475 -1.12369785]
  [ 1.88304891 -1.56798295 -0.41669772 -0.47413281]
  [-0.27787888 -0.38984243 -0.27826104  0.77916954]
  [-1.8527932  -0.22046015  0.35280648 -2.09924956]
  [-2.31658839  2.00762315 -1.41188379 -0.59375061]]

 [[-0.53692297 -0.6014518   0.47550973 -0.97483527]
  [-0.04411031  0.66553113  1.47607725  0.74410961]
  [-1.09388844  1.86195703 -0.25840725 -1.81813154]
  [ 1.10489318  0.66724034  0.55600656  2.75490493]
  [ 0.9024805  -1.3117066   1.16503191 -0.95704286]]]

hs_dec= [[[ 0.39436226  0.9989428   1.49135418  0.43299438]
  [-0.88883041  0.02623611 -0.4730215   0.32916778]
  [-0.15123933  2.3083466   1.39889384 -2.15295063]
  [ 0.66861353 -0.50004443  1.07123667  0